In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./groceries/train.csv")

In [3]:
data

,image_id,label
0,train_1a,rice
1,train_1b,candy
2,train_1c,jam
3,train_1d,coffee
4,train_2a,vinegar
...,...,...
3210,train_803c,soda
3211,train_803d,spices
3212,train_804a,nuts
3213,train_804b,candy


In [5]:
labels = data.label.unique()

In [6]:
labels

array(['rice', 'candy', 'jam', 'coffee', 'vinegar', 'chocolate', 'sugar',
       'water', 'juice', 'milk', 'soda', 'nuts', 'chips', 'spices',
       'cereal', 'beans', 'cake', 'honey', 'flour', 'pasta',
       'tomatosauce', 'tea', 'corn', 'oil', 'fish'], dtype=object)

In [7]:
len(labels)

25

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3215 entries, 0 to 3214
Data columns (total 2 columns):
image_id    3215 non-null object
label       3215 non-null object
dtypes: object(2)
memory usage: 50.4+ KB


In [11]:
import os
os.mkdir("./groceries/train/")
for label in labels:
    os.mkdir("./groceries/train/"+label)

In [16]:
data.head()

,image_id,label
0,train_1a,rice
1,train_1b,candy
2,train_1c,jam
3,train_1d,coffee
4,train_2a,vinegar


In [12]:
import shutil

In [28]:
source = "./groceries/train_img/"
destination = "./groceries/train/"
for index,row in data.iterrows():
    print(row['image_id'],row['label'])
    print(source+row['image_id'])
    print(destination+row['label']+'/'+row['image_id'])
    shutil.move(source+row['image_id']+'.png', destination+row['label']+'/'+row['image_id']+'.png') 

train_1a rice
./groceries/train_img/train_1a
./groceries/train/rice/train_1a
train_1b candy
./groceries/train_img/train_1b
./groceries/train/candy/train_1b
train_1c jam
./groceries/train_img/train_1c
./groceries/train/jam/train_1c
train_1d coffee
./groceries/train_img/train_1d
./groceries/train/coffee/train_1d
train_2a vinegar
./groceries/train_img/train_2a
./groceries/train/vinegar/train_2a
train_2b chocolate
./groceries/train_img/train_2b
./groceries/train/chocolate/train_2b
train_2c sugar
./groceries/train_img/train_2c
./groceries/train/sugar/train_2c
train_2d water
./groceries/train_img/train_2d
./groceries/train/water/train_2d
train_3a juice
./groceries/train_img/train_3a
./groceries/train/juice/train_3a
train_3b candy
./groceries/train_img/train_3b
./groceries/train/candy/train_3b
train_3c juice
./groceries/train_img/train_3c
./groceries/train/juice/train_3c
train_3d juice
./groceries/train_img/train_3d
./groceries/train/juice/train_3d
train_4a candy
./groceries/train_img/train_4

In [29]:
# entrenamos un clasificador puesto que tenemos las imagenes etiquetadas
# no hay imagenes de test
# usamos un clasificador
# usar una red ya hecha como resnet y la puedo entrenar desde cero o usar transferlearning
# Al tener pocos datos usamos https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# Podemos aplicar modificaciones a las imagenes para ampliar datos de entrenamiento

In [44]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras import optimizers

In [35]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

58892288/58889256 [==============================] - 47s 1us/step


In [36]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [47]:
# dimensions of our images.
img_width, img_height = 150, 150
train_data_dir = "./groceries/train/"
validation_data_dir = './groceries/train'
epochs = 50
batch_size = 16

#mejor un modelo resnet50 para que sea más facil entrenar
model = applications.resnet_v2.ResNet50V2(weights='imagenet', include_top=False) #Al poner falso la descabezo

94674944/94668760 [==============================] - 354s 4us/step


In [50]:
model.summary()

#no entreno el modelo ya entrenado
for layer in model.layers:
    layer.trainable = False

# build a classifier model to put on top of the convolutional model, que es un clasificador de 25 clases de salida
top_model = Sequential()
#top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(25, activation='softmax')) #capa de salida

# add the model on top of the convolutional base
model.add(top_model)

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, None, None, 6 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

NameError: name 'top_model_weights_path' is not defined

In [57]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# fine-tune the model
nb_validation_samples = 3216
nb_train_samples = 3216

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 3216 images belonging to 25 classes.
Found 3216 images belonging to 25 classes.


TypeError: fit_generator() got an unexpected keyword argument 'samples_per_epoch'